## Embeddings

Sa ating nakaraang halimbawa, gumamit tayo ng high-dimensional na bag-of-words vectors na may haba na `vocab_size`, at tahasang kinonvert ang low-dimensional positional representation vectors sa sparse one-hot representation. Ang one-hot representation na ito ay hindi memory-efficient. Bukod dito, ang bawat salita ay itinuturing na hiwalay sa isa't isa, kaya't ang one-hot encoded vectors ay hindi nagpapakita ng semantikong pagkakatulad sa pagitan ng mga salita.

Sa unit na ito, ipagpapatuloy natin ang pag-explore sa **News AG** dataset. Upang magsimula, i-load natin ang data at kunin ang ilang mga depinisyon mula sa nakaraang unit.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Ano ang embedding?

Ang ideya ng **embedding** ay ang pagrepresenta ng mga salita gamit ang mas mababang-dimensional na dense vectors na sumasalamin sa semantikong kahulugan ng salita. Tatalakayin natin mamaya kung paano bumuo ng makabuluhang word embeddings, pero sa ngayon, isipin muna natin ang embeddings bilang isang paraan upang bawasan ang dimensionality ng isang word vector.

Kaya, ang isang embedding layer ay tumatanggap ng isang salita bilang input, at naglalabas ng output vector na may tinukoy na `embedding_size`. Sa isang banda, ito ay halos katulad ng isang `Dense` layer, ngunit sa halip na tumanggap ng one-hot encoded vector bilang input, kaya nitong tumanggap ng word number.

Sa pamamagitan ng paggamit ng embedding layer bilang unang layer sa ating network, maaari tayong lumipat mula sa bag-of-words patungo sa isang **embedding bag** model, kung saan una nating kino-convert ang bawat salita sa ating teksto sa kaukulang embedding, at pagkatapos ay kinakalkula ang isang aggregate function sa lahat ng mga embeddings na iyon, tulad ng `sum`, `average`, o `max`.

![Larawan na nagpapakita ng isang embedding classifier para sa limang sequence na salita.](../../../../../translated_images/tl/embedding-classifier-example.b77f021a7ee67eee.webp)

Ang ating classifier neural network ay binubuo ng mga sumusunod na layer:

* `TextVectorization` layer, na tumatanggap ng string bilang input, at naglalabas ng tensor ng mga token numbers. Magtatakda tayo ng isang makatwirang laki ng bokabularyo na `vocab_size`, at babalewalain ang mga salitang hindi madalas gamitin. Ang input shape ay magiging 1, at ang output shape ay magiging $n$, dahil makakakuha tayo ng $n$ tokens bilang resulta, bawat isa ay naglalaman ng mga numero mula 0 hanggang `vocab_size`.
* `Embedding` layer, na tumatanggap ng $n$ na numero, at binabawasan ang bawat numero sa isang dense vector na may tinukoy na haba (100 sa ating halimbawa). Kaya, ang input tensor na may shape na $n$ ay mababago sa isang $n\times 100$ tensor.
* Aggregation layer, na kumukuha ng average ng tensor na ito sa kahabaan ng unang axis, i.e., kakalkulahin nito ang average ng lahat ng $n$ input tensors na tumutugma sa iba't ibang salita. Upang ipatupad ang layer na ito, gagamit tayo ng isang `Lambda` layer, at ipapasa dito ang function upang kalkulahin ang average. Ang output ay magkakaroon ng shape na 100, at ito ang magiging numerikal na representasyon ng buong input sequence.
* Panghuling `Dense` linear classifier.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


Sa `summary` na printout, sa **output shape** na column, ang unang dimensyon ng tensor na `None` ay tumutukoy sa laki ng minibatch, at ang pangalawa ay tumutukoy sa haba ng token sequence. Ang lahat ng token sequences sa minibatch ay may iba't ibang haba. Tatalakayin natin kung paano ito haharapin sa susunod na seksyon.

Ngayon, simulan na natin ang pag-train ng network:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Tandaan** na gumagawa tayo ng vectorizer batay sa isang subset ng data. Ginagawa ito upang mapabilis ang proseso, at maaaring magresulta ito sa sitwasyon kung saan hindi lahat ng token mula sa ating teksto ay nasa bokabularyo. Sa ganitong kaso, ang mga token na iyon ay hindi isasama, na maaaring magresulta sa bahagyang mas mababang katumpakan. Gayunpaman, sa totoong buhay, ang isang subset ng teksto ay madalas na nagbibigay ng magandang pagtataya ng bokabularyo.


### Pagharap sa iba't ibang laki ng sequence ng variable

Unawain natin kung paano nangyayari ang training sa minibatches. Sa halimbawa sa itaas, ang input tensor ay may dimensyong 1, at gumagamit tayo ng 128-long minibatches, kaya ang aktwal na laki ng tensor ay $128 \times 1$. Gayunpaman, ang bilang ng mga token sa bawat pangungusap ay magkakaiba. Kung gagamitin natin ang `TextVectorization` layer sa isang input, ang bilang ng mga token na ibinabalik ay iba-iba, depende sa kung paano tinokenize ang teksto:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Gayunpaman, kapag inilapat natin ang vectorizer sa ilang mga sequence, kailangan nitong gumawa ng tensor na may hugis na parihaba, kaya pinupunan nito ang mga hindi nagamit na elemento gamit ang PAD token (na sa ating kaso ay zero):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Narito makikita natin ang mga embeddings:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Tandaan**: Upang mabawasan ang dami ng padding, sa ilang mga kaso makatuwiran na ayusin ang lahat ng mga sequence sa dataset ayon sa pagtaas ng haba (o, mas tiyak, bilang ng mga token). Ito ay magtitiyak na ang bawat minibatch ay naglalaman ng mga sequence na may magkakatulad na haba.


## Semantic embeddings: Word2Vec

Sa ating nakaraang halimbawa, ang embedding layer ay natutong i-map ang mga salita sa vector representations, ngunit ang mga representasyong ito ay walang semantikong kahulugan. Maganda sana kung makakagawa tayo ng vector representation kung saan ang magkatulad na salita o mga kasingkahulugan ay tumutugma sa mga vectors na malapit sa isa't isa batay sa ilang uri ng vector distance (halimbawa, euclidian distance).

Upang magawa ito, kailangan nating i-pretrain ang ating embedding model sa isang malaking koleksyon ng teksto gamit ang isang teknik tulad ng [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Ito ay nakabatay sa dalawang pangunahing arkitektura na ginagamit upang makabuo ng distributed representation ng mga salita:

 - **Continuous bag-of-words** (CBoW), kung saan tinetrain natin ang modelo upang hulaan ang isang salita mula sa nakapaligid na konteksto. Ibinigay ang ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, ang layunin ng modelo ay hulaan ang $W_0$ mula sa $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** ay kabaligtaran ng CBoW. Ginagamit ng modelo ang nakapaligid na window ng mga salita sa konteksto upang hulaan ang kasalukuyang salita.

Mas mabilis ang CBoW, ngunit habang mas mabagal ang skip-gram, mas mahusay ito sa pagrepresenta ng mga bihirang salita.

![Larawan na nagpapakita ng parehong CBoW at Skip-Gram na mga algorithm para i-convert ang mga salita sa vectors.](../../../../../translated_images/tl/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Upang mag-eksperimento gamit ang Word2Vec embedding na pre-trained sa Google News dataset, maaari nating gamitin ang **gensim** library. Sa ibaba, makikita natin ang mga salitang pinakamalapit sa 'neural'.

> **Note:** Kapag unang gumawa ng word vectors, maaaring tumagal ang pag-download ng mga ito!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Maaari rin nating kunin ang vector embedding mula sa salita, upang magamit sa pagsasanay ng classification model. Ang embedding ay may 300 na bahagi, ngunit dito ipinapakita lamang natin ang unang 20 bahagi ng vector para sa kalinawan:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Ang mahusay na bagay tungkol sa semantic embeddings ay maaari mong manipulahin ang vector encoding batay sa semantika. Halimbawa, maaari nating hanapin ang isang salita na ang vector representation ay pinakamalapit sa mga salitang *hari* at *babae*, at pinakamalayo sa salitang *lalaki*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Ang halimbawa sa itaas ay gumagamit ng ilang panloob na GenSym magic, ngunit ang pangunahing lohika ay talagang medyo simple. Isang kawili-wiling bagay tungkol sa embeddings ay maaari kang magsagawa ng normal na operasyon ng vector sa embedding vectors, at iyon ay magpapakita ng mga operasyon sa mga **kahulugan** ng salita. Ang halimbawa sa itaas ay maaaring ipahayag sa mga termino ng operasyon ng vector: kinakalkula natin ang vector na tumutugma sa **HARI-LALAKI+BABAE** (ang mga operasyon na `+` at `-` ay isinasagawa sa mga representasyon ng vector ng mga kaukulang salita), at pagkatapos ay hinahanap ang pinakamalapit na salita sa diksyunaryo sa vector na iyon:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Kinailangan naming magdagdag ng maliit na coefficients sa *man* at *woman* na mga vector - subukang alisin ito upang makita ang mangyayari.

Upang mahanap ang pinakamalapit na vector, ginagamit namin ang TensorFlow machinery upang kalkulahin ang isang vector ng mga distansya sa pagitan ng aming vector at lahat ng mga vector sa bokabularyo, at pagkatapos ay hanapin ang index ng pinakamaliit na salita gamit ang `argmin`.


Habang ang Word2Vec ay mukhang isang mahusay na paraan upang ipahayag ang semantika ng mga salita, mayroon itong maraming mga kahinaan, kabilang ang mga sumusunod:

* Parehong CBoW at skip-gram na mga modelo ay **predictive embeddings**, at isinasaalang-alang lamang nila ang lokal na konteksto. Hindi ginagamit ng Word2Vec ang global na konteksto.
* Hindi isinasaalang-alang ng Word2Vec ang **morpolohiya** ng salita, ibig sabihin, ang katotohanan na ang kahulugan ng salita ay maaaring nakadepende sa iba't ibang bahagi nito, tulad ng ugat.

Sinusubukan ng **FastText** na malampasan ang ikalawang limitasyon, at pinapahusay ang Word2Vec sa pamamagitan ng pag-aaral ng mga vector na representasyon para sa bawat salita at ang mga character n-grams na matatagpuan sa loob ng bawat salita. Ang mga halaga ng mga representasyon ay pagkatapos ay ina-average sa isang vector sa bawat hakbang ng pagsasanay. Bagama't nagdadagdag ito ng maraming karagdagang pagkalkula sa pretraining, pinapahintulutan nito ang word embeddings na mag-encode ng impormasyon sa sub-word.

Ang isa pang pamamaraan, **GloVe**, ay gumagamit ng ibang diskarte sa word embeddings, na nakabatay sa factorization ng word-context matrix. Una, bumubuo ito ng isang malaking matrix na nagbibilang ng dami ng paglitaw ng mga salita sa iba't ibang konteksto, at pagkatapos ay sinusubukan nitong i-representa ang matrix na ito sa mas mababang dimensyon sa paraang nagpapaliit ng reconstruction loss.

Sinusuportahan ng gensim library ang mga word embeddings na ito, at maaari kang mag-eksperimento sa mga ito sa pamamagitan ng pagbabago ng model loading code sa itaas.


## Paggamit ng pretrained embeddings sa Keras

Maaari nating baguhin ang halimbawa sa itaas upang punan ang matrix sa ating embedding layer gamit ang semantic embeddings, tulad ng Word2Vec. Malamang na hindi magtutugma ang mga bokabularyo ng pretrained embedding at ng text corpus, kaya kailangan nating pumili ng isa. Dito natin susuriin ang dalawang posibleng opsyon: paggamit ng bokabularyo ng tokenizer, at paggamit ng bokabularyo mula sa Word2Vec embeddings.

### Paggamit ng bokabularyo ng tokenizer

Kapag ginamit ang bokabularyo ng tokenizer, ang ilan sa mga salita mula sa bokabularyo ay magkakaroon ng kaukulang Word2Vec embeddings, at ang ilan ay mawawala. Dahil ang laki ng ating bokabularyo ay `vocab_size`, at ang haba ng Word2Vec embedding vector ay `embed_size`, ang embedding layer ay kakatawanin ng isang weight matrix na may hugis na `vocab_size`$\times$`embed_size`. Pupunan natin ang matrix na ito sa pamamagitan ng pagdaan sa bokabularyo:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Para sa mga salitang wala sa Word2Vec vocabulary, maaari nating iwanan ang mga ito bilang mga zero, o bumuo ng random na vector.

Ngayon, maaari na tayong magtakda ng embedding layer na may pretrained na mga timbang:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Tandaan**: Pansinin na itinakda natin ang `trainable=False` kapag nilikha ang `Embedding`, na nangangahulugang hindi natin muling sinasanay ang Embedding layer. Maaaring bahagyang bumaba ang katumpakan nito, ngunit mas pinapabilis nito ang pagsasanay.

### Paggamit ng bokabularyo ng embedding

Isang isyu sa naunang paraan ay ang pagkakaiba ng mga bokabularyo na ginamit sa TextVectorization at Embedding. Upang malampasan ang problemang ito, maaari nating gamitin ang isa sa mga sumusunod na solusyon:
* Muling sanayin ang Word2Vec model gamit ang ating bokabularyo.
* I-load ang ating dataset gamit ang bokabularyo mula sa pretrained na Word2Vec model. Ang mga bokabularyong gagamitin para i-load ang dataset ay maaaring tukuyin habang naglo-load.

Ang huling paraan ay tila mas madali, kaya't ipapatupad natin ito. Una sa lahat, lilikha tayo ng isang `TextVectorization` layer gamit ang tinukoy na bokabularyo, na kinuha mula sa Word2Vec embeddings:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Ang gensim word embeddings library ay naglalaman ng isang maginhawang function, `get_keras_embeddings`, na awtomatikong lilikha ng kaukulang Keras embeddings layer para sa iyo.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Isa sa mga dahilan kung bakit hindi natin nakikita ang mas mataas na katumpakan ay dahil ang ilang mga salita mula sa aming dataset ay nawawala sa pretrained na bokabularyo ng GloVe, kaya't sila ay karaniwang hindi pinapansin. Upang malampasan ito, maaari tayong mag-train ng sarili nating mga embeddings batay sa aming dataset.


## Mga Kontekstwal na Embedding

Isa sa mga pangunahing limitasyon ng tradisyunal na pretrained embedding representations tulad ng Word2Vec ay ang katotohanang, kahit na kaya nilang makuha ang ilang kahulugan ng isang salita, hindi nila kayang pag-iba-ibahin ang magkakaibang kahulugan nito. Maaari itong magdulot ng mga problema sa mga downstream na modelo.

Halimbawa, ang salitang 'play' ay may iba't ibang kahulugan sa dalawang magkaibang pangungusap na ito:
- Pumunta ako sa isang **play** sa teatro.
- Gusto ni John na **maglaro** kasama ang kanyang mga kaibigan.

Ang mga pretrained embeddings na nabanggit natin ay kumakatawan sa parehong kahulugan ng salitang 'play' sa iisang embedding. Upang malampasan ang limitasyong ito, kailangan nating bumuo ng mga embedding batay sa **language model**, na sinanay sa isang malaking corpus ng teksto, at *alam* kung paano maaaring pagsama-samahin ang mga salita sa iba't ibang konteksto. Ang pagtalakay sa mga kontekstwal na embedding ay lampas sa saklaw ng tutorial na ito, ngunit babalikan natin ito kapag pinag-usapan na ang mga language model sa susunod na yunit.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi eksaktong salin. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na pinagmulan. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
